In [2]:
import numpy as np
import cv2 as cv
import glob
import os

# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(7,5,0)
objp = np.zeros((6*9, 3), np.float32)
objp[:, :2] = np.mgrid[0:9, 0:6].T.reshape(-1, 2)

# Arrays to store object points and image points from all the images.
objpoints = []  # 3d point in real world space
imgpoints = []  # 2d points in image plane.

images = glob.glob('/home/user/teamA-2/fullfilment/src/amr_control/images/turtlebot_cam/*.png')

for fname in images:
    img = cv.imread(fname)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

    # Find the chess board corners
    ret, corners = cv.findChessboardCorners(gray, (9, 6), None)

    # If found, add object points, image points (after refining them)
    if ret:
        objpoints.append(objp)

        corners2 = cv.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
        imgpoints.append(corners2)

        # Draw and display the corners
        cv.drawChessboardCorners(img, (9, 6), corners2, ret)
        cv.imshow('img', img)
        cv.waitKey(500)
    else:
        print(f"Chessboard corners not found in image: {fname}")

cv.destroyAllWindows()

# Perform camera calibration
if objpoints and imgpoints:
    ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

    # Print calibration results
    print("Camera matrix:")
    print(mtx)
    print("\nDistortion coefficients:")
    print(dist)

    # File paths
    mtx_file = '//home/user/teamA-2/fullfilment/src/amr_control/images/turtlebot_cam/camera_matrix.txt'
    dist_file = '/home/user/teamA-2/fullfilment/src/amr_control/images/turtlebot_cam/dist_coeffs.txt'

    # Check if files exist and update or create them
    if os.path.exists(mtx_file):
        print(f"Updating {mtx_file}")
    else:
        print(f"Creating {mtx_file}")

    if os.path.exists(dist_file):
        print(f"Updating {dist_file}")
    else:
        print(f"Creating {dist_file}")

    # Save the camera matrix and distortion coefficients
    np.savetxt(mtx_file, mtx, delimiter=',', header='Camera matrix', comments='')
    np.savetxt(dist_file, dist, delimiter=',', header='Distortion coefficients', comments='')

else:
    print("Not enough valid images for calibration.")


Camera matrix:
[[1.38186935e+03 0.00000000e+00 6.90405953e+02]
 [0.00000000e+00 1.38798341e+03 3.90416171e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]

Distortion coefficients:
[[ 0.02311391  0.3674243  -0.0109465  -0.00204379 -1.38311693]]
Updating //home/user/teamA-2/fullfilment/src/amr_control/images/global_cam/camera_matrix.txt
Updating /home/user/teamA-2/fullfilment/src/amr_control/images/global_cam/dist_coeffs.txt


In [3]:
ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

In [5]:
img = cv.imread('Fisheye1_11.jpg')
h,  w = img.shape[:2]
newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))

[ WARN:0@101.810] global loadsave.cpp:241 findDecoder imread_('Fisheye1_11.jpg'): can't open/read file: check file path/integrity


AttributeError: 'NoneType' object has no attribute 'shape'

In [ ]:
# undistort
dst = cv.undistort(img, mtx, dist, None, newcameramtx)

# crop the image
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]
cv.imwrite('calibresult.png', dst)

In [ ]:
# undistort
mapx, mapy = cv.initUndistortRectifyMap(mtx, dist, None, newcameramtx, (w,h), 5)
dst = cv.remap(img, mapx, mapy, cv.INTER_LINEAR)

# crop the image
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]
cv.imwrite('calibresult.png', dst)

In [ ]:
mean_error = 0
for i in range(len(objpoints)):
    imgpoints2, _ = cv.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
    error = cv.norm(imgpoints[i], imgpoints2, cv.NORM_L2)/len(imgpoints2)
    mean_error += error

print( "total error: {}".format(mean_error/len(objpoints)) )